## Finding Fake/Duplicate Records
Commonly known as Data Dedupication & Record Linkage
Ref: https://www.kaggle.com/code/caesarlupum/deduping-record-linkage

In [ ]:
!pip install nltk

In [ ]:
import nltk
nltk.download('stopwords')
nltktk.download('punkt')

In [2]:
from datetime import datetime, date
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import string
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import FeatureUnion, Pipeline
import re
import string
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer # or LancasterStemmer, RegexpStemmer, SnowballStemmer

default_stemmer = PorterStemmer()
default_stopwords = stopwords.words('english') # or any other list of your choice

%matplotlib inline

In [64]:
df = pd.read_csv('../data/listings.csv')

In [65]:
df.head()

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,528022,https://www.airbnb.com/rooms/528022,20221205143541,2022-12-05,city scrape,Somewhere Else,Highlights<br />*Located only 5 minutes from C...,NaN,https://a0.muscache.com/pictures/bf6fa79c-5863...,2594559,...,4.90,4.78,4.72,0363 9289 A94D 5C21 A579,f,1,0,1,0,3.30
1,2818,https://www.airbnb.com/rooms/2818,20221205143541,2022-12-05,city scrape,Quiet Garden View Room & Super Fast Wi-Fi,Quiet Garden View Room & Super Fast Wi-Fi<br /...,"Indische Buurt (""Indies Neighborhood"") is a ne...",https://a0.muscache.com/pictures/10272854/8dcc...,3159,...,4.98,4.69,4.81,0363 5F3A 5684 6750 D14D,f,1,0,1,0,1.88
2,20168,https://www.airbnb.com/rooms/20168,20221205143541,2022-12-05,previous scrape,Studio with private bathroom in the centre 1,17th century Dutch townhouse in the heart of t...,Located just in between famous central canals....,https://a0.muscache.com/pictures/69979628/fd6a...,59484,...,4.62,4.87,4.49,0363 CBB3 2C10 0C2A 1E29,t,2,0,2,0,2.18
3,27886,https://www.airbnb.com/rooms/27886,20221205143541,2022-12-05,city scrape,"Romantic, stylish B&B houseboat in canal district",Stylish and romantic houseboat on fantastic hi...,"Central, quiet, safe, clean and beautiful.",https://a0.muscache.com/pictures/02c2da9d-660e...,97647,...,4.92,4.89,4.79,0363 974D 4986 7411 88D8,t,1,0,1,0,1.83
4,28871,https://www.airbnb.com/rooms/28871,20221205143541,2022-12-05,city scrape,Comfortable double room,<b>The space</b><br />In a monumental house ri...,"Flower market , Leidseplein , Rembrantsplein",https://a0.muscache.com/pictures/160889/362340...,124245,...,4.94,4.97,4.83,0363 607B EA74 0BD8 2F6F,f,2,0,2,0,3.03


In [67]:
# Since this is a data deduplication problem, we're interested in data uploaded by the host, let's print list of columns to find out those columns
df.columns

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'source', 'name',
       'description', 'neighborhood_overview', 'picture_url', 'host_id',
       'host_url', 'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 'ca

In [72]:
df['host_location'].unique()

array(['Amsterdam, Netherlands', 'Netherlands',
       'Nieuw Sloten, Netherlands', 'New York, NY',
       'London, United Kingdom', 'De Pijp, Netherlands',
       'La Cala de Mijas, Spain', 'Heerhugowaard, Netherlands',
       'Haarlem, Netherlands', nan, 'Warmond, Netherlands',
       'The Hague, Netherlands', 'Zandvoort, Netherlands',
       'Harlingen, Netherlands', 'Berlin, Germany',
       'Enkhuizen, Netherlands', 'Bentveld, Netherlands',
       'Washington, DC', 'Hilversum, Netherlands',
       'Nijmegen, Netherlands', 'Nieuwendam, Netherlands',
       'Heemskerk, Netherlands', 'Chicago, IL', 'Brasov, Romania',
       'Katwoude, Netherlands', 'Nederland, CO', 'Leusden, Netherlands',
       'Nieuwegein, Netherlands', 'Amstelveen, Netherlands',
       'Sydney, Australia', 'Paris, France', 'Vinkeveen, Netherlands',
       'Bergen, Netherlands', 'Eemnes, Netherlands',
       'Schoorl, Netherlands', 'Malaysia', 'İstanbul, Turkey',
       'Bogotá, Colombia', 'Germany', 'Hamburg, Germ

In [ ]:
selectedcols = ['name', 'description', 'neighborhood_overview', 'host_id',
       'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'has_availability',
       'availability_30', 'availability_60', 'availability_90',
       'availability_365']

In [ ]:
df = df[selectedcols]
df.head()

In [ ]:
# Now lets first understand and work on numerical columns
df.describe()

In [ ]:
# drop nan columns
nan_cols = ['bathrooms',
           'neighbourhood_group_cleansed']
df.drop(nan_cols, axis=1, inplace=True)
df.describe()

In [ ]:
# Get list of text columns
objcols = df.select_dtypes('object').columns
objcols

In [ ]:
# Lets drop picture urls
drop_cols = ['host_thumbnail_url', 'host_picture_url']
df.drop(drop_cols, axis=1, inplace=True)

In [ ]:
# host_name
df['host_name'].str.lower().value_counts()

In [ ]:
no = df['neighborhood_overview'].astype('str')
len(no[0])

In [ ]:
# host_since
def age(born):
    born = datetime.strptime(born, "%Y-%m-%d").date()
    today = date.today()
    return today.year - born.year - ((today.month, 
                                      today.day) < (born.month, 
                                                    born.day))
df['age'] = df['host_since'].apply(age)
df['age']
df.drop('host_since', axis=1, inplace=True)
df.head()

In [ ]:
# host_location, Most of the values are similar so can be dropped
df['host_location'].value_counts()
df.drop('host_location', axis=1, inplace=True)

In [ ]:
# host_about

In [ ]:
# host_response_time
df['host_response_time'].value_counts()
df['host_response_time'].replace(['within an hour', 'within a few hours', 'within a day', 'a few days or more'],
                                [1, 2, 3, 4], inplace=True)
df['host_response_time'] = df['host_response_time'].astype('int64')

In [ ]:
# Lot of null values are there, but this can be important depictor of host behaviour and property, so let's impute
df['host_response_time'].isnull().value_counts()

In [ ]:
sns.boxplot(df.host_response_time)
#
# Distribution plot
#
sns.distplot(df.host_response_time)

In [ ]:
# Lets fill with median value
df['host_response_time'] = df['host_response_time'].fillna(df['host_response_time'].median())
df['host_response_time'].isnull().value_counts()

In [ ]:
# host_response_rate
df['host_response_rate'].isnull().value_counts()

In [ ]:
# host_acceptance_rate

In [ ]:
# host_is_superhost

## Let's create a simple dataset from property columns

In [5]:
selectedcols = ['name', 'description', 'neighborhood_overview', 'neighbourhood',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'has_availability',
       'availability_30', 'availability_60', 'availability_90',
       'availability_365']

In [6]:
df = df[selectedcols]
df.head()

,name,description,neighborhood_overview,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,property_type,room_type,...,beds,amenities,price,minimum_nights,maximum_nights,has_availability,availability_30,availability_60,availability_90,availability_365
0,Somewhere Else,Highlights<br />*Located only 5 minutes from C...,NaN,NaN,Westerpark,NaN,52.39114,4.88846,Private room in boat,Private room,...,1.0,"[""Bed linens"", ""Dishes and silverware"", ""Hange...",$100.00,2,30,t,4,16,29,212
1,Quiet Garden View Room & Super Fast Wi-Fi,Quiet Garden View Room & Super Fast Wi-Fi<br /...,"Indische Buurt (""Indies Neighborhood"") is a ne...","Amsterdam, North Holland, Netherlands",Oostelijk Havengebied - Indische Buurt,NaN,52.36435,4.94358,Private room in rental unit,Private room,...,2.0,"[""Books and reading material"", ""Smoke alarm"", ...",$59.00,3,28,t,6,27,27,27
2,Studio with private bathroom in the centre 1,17th century Dutch townhouse in the heart of t...,Located just in between famous central canals....,"Amsterdam, North Holland, Netherlands",Centrum-Oost,NaN,52.36407,4.89393,Private room in townhouse,Private room,...,1.0,"[""Smoke alarm"", ""Bed linens"", ""Hangers"", ""Wifi...",$106.00,1,365,t,0,0,0,0
3,"Romantic, stylish B&B houseboat in canal district",Stylish and romantic houseboat on fantastic hi...,"Central, quiet, safe, clean and beautiful.","Amsterdam, North Holland, Netherlands",Centrum-West,NaN,52.38761,4.89188,Private room in houseboat,Private room,...,1.0,"[""Books and reading material"", ""Smoke alarm"", ...",$140.00,2,356,t,0,17,32,40
4,Comfortable double room,<b>The space</b><br />In a monumental house ri...,"Flower market , Leidseplein , Rembrantsplein","Amsterdam, North Holland, Netherlands",Centrum-West,NaN,52.36775,4.89092,Private room in rental unit,Private room,...,1.0,"[""Smoke alarm"", ""Bed linens"", ""Lock on bedroom...",$75.00,2,1825,t,0,3,8,91


In [7]:
df.isna().sum()

name                               0
description                        8
neighborhood_overview           2381
neighbourhood                   2381
neighbourhood_cleansed             0
neighbourhood_group_cleansed    6809
latitude                           0
longitude                          0
property_type                      0
room_type                          0
accommodates                       0
bathrooms                       6809
bathrooms_text                    13
bedrooms                         286
beds                              91
amenities                          0
price                              0
minimum_nights                     0
maximum_nights                     0
has_availability                   0
availability_30                    0
availability_60                    0
availability_90                    0
availability_365                   0
dtype: int64

In [8]:
# drop nan columns
nan_cols = ['bathrooms',
           'neighbourhood_group_cleansed']
df.drop(nan_cols, axis=1, inplace=True)

In [9]:
# Almost 40-50% rows in neighborhood_overview, neighborhood so lets drop them
nan_cols = ['neighborhood_overview',
           'neighbourhood']
df.drop(nan_cols, axis=1, inplace=True)

In [10]:
df.isna().sum()

name                        0
description                 8
neighbourhood_cleansed      0
latitude                    0
longitude                   0
property_type               0
room_type                   0
accommodates                0
bathrooms_text             13
bedrooms                  286
beds                       91
amenities                   0
price                       0
minimum_nights              0
maximum_nights              0
has_availability            0
availability_30             0
availability_60             0
availability_90             0
availability_365            0
dtype: int64

In [11]:
# Fillna
df['description'].fillna('', inplace=True)
df['bathrooms_text'].fillna('', inplace=True)
df['bedrooms'].fillna(df['bedrooms'].mean(), inplace=True)
df['beds'].fillna(df['beds'].mean(), inplace=True)

In [ ]:
def ngrams(text, n=3):
    text = text.encode("ascii", errors="ignore").decode() #remove non ascii chars
    text = text.lower()
    text = text.replace('&', ' and ')
    text = text.replace(',', ' ')
    text = text.replace('-', ' ')
#     chars_to_remove = [")","(",".","|","[","]","{","}","'"]
#     rx = '[' + re.escape(''.join(chars_to_remove)) + ']'
#     text = re.sub(rx, '', text)
    text = [words for words in text if words not in string.punctuation]
    text = ' '.join(text)
    text = re.sub(' +',' ',text).strip() # get rid of multiple spaces and replace with a single
    text = ' '+ text +' ' # pad names for ngrams...
    text = re.sub(r'[,-./]|\sBD',r'', text)
    ngrams = zip(*[text[i:] for i in range(n)])
    return [''.join(ngram) for ngram in ngrams]


In [ ]:
ngrams("Hello")

In [ ]:
property_names = df['name'].unique()
vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams)
tf_idf_matrix = vectorizer.fit_transform(property_names)

In [ ]:
tf_idf_matrix.shape

In [47]:
clean_text(df.iloc[0]['description'])

'highlight locat onli 5 minut citi center short distanc central station fast free wifi doubl bed uniqu houseboat space somewher els bed breakfast beauti old ship ‘ cornelia ’ locat oud houthaven 10 minut walk distanc form central station find thi charm b b privat guest accommod old skipper ’ quarter entranc wheelhous side ’ wide view water even toilet thi bed breakfast reflect ultim amsterdam experi friendli atmospher area almost feel like villag thi properti situat old wood harbour onli short walk amsterdam canal jordaan surround first time hop aboard “ cornelia ” houseboat steep histori built 1905 immedi feel relax home co'

In [46]:
CLEANR = re.compile('<.*?>') 

def clean_series(inp_series):
    result = inp_series.apply(lambda x: clean_text(x))
    return result
    
def clean_text(text, ):

    def tokenize_text(text):
        return [w for s in sent_tokenize(text) for w in word_tokenize(s)]
    
    def remove_html_tags(text):
        return re.sub(CLEANR, '', text)

    def remove_special_characters(text, characters=string.punctuation.replace('-', '')):
        tokens = tokenize_text(text)
        pattern = re.compile('[{}]'.format(re.escape(characters)))
        return ' '.join(filter(None, [pattern.sub('', t) for t in tokens]))

    def stem_text(text, stemmer=default_stemmer):
        tokens = tokenize_text(text)
        return ' '.join([stemmer.stem(t) for t in tokens])

    def remove_stopwords(text, stop_words=default_stopwords):
        tokens = [w for w in tokenize_text(text) if w not in stop_words]
        return ' '.join(tokens)

    text = text.strip(' ') # strip whitespaces
    text = text.lower() # lowercase
    text = stem_text(text) # stemming
    text = remove_html_tags(text)
    text = remove_special_characters(text) # remove punctuation and symbols
    text = remove_stopwords(text) # remove stopwords
    #text.strip(' ') # strip whitespaces again?

    return text

In [13]:
transformer = FeatureUnion([
                ('name_tfidf', 
                  Pipeline([('extract_field',
                              FunctionTransformer(lambda x: x['name'], 
                                                  validate=False)),
                            ('cleaning', 
                              FunctionTransformer(clean_series)),
                            ('tfidf', 
                              TfidfVectorizer())])),
                ('description_tfidf', 
                  Pipeline([('extract_field', 
                              FunctionTransformer(lambda x: x['description'], 
                                                  validate=False)),
                            ('cleaning', 
                              FunctionTransformer(clean_series)),
                            ('tfidf', 
                              TfidfVectorizer())]))]) 

transformer.fit(df)



FeatureUnion(transformer_list=[('name_tfidf',
                                Pipeline(steps=[('extract_field',
                                                 FunctionTransformer(func=<function <lambda> at 0x7fa480ff0430>)),
                                                ('cleaning',
                                                 FunctionTransformer(func=<function clean_series at 0x7fa480ff01f0>)),
                                                ('tfidf', TfidfVectorizer())])),
                               ('description_tfidf',
                                Pipeline(steps=[('extract_field',
                                                 FunctionTransformer(func=<function <lambda> at 0x7fa480ff04c0>)),
                                                ('cleaning',
                                                 FunctionTransformer(func=<function clean_series at 0x7fa480ff01f0>)),
                                                ('tfidf',
                                        

In [ ]:
search_vocab = transformer.transformer_list[0][1].steps[2][1].get_feature_names() 
product_vocab = transformer.transformer_list[1][1].steps[2][1].get_feature_names()
vocab = search_vocab + product_vocab

print(vocab)

In [14]:
tfidf_matrix = transformer.transform(df).toarray()

In [15]:
print( tfidf_matrix.shape, tfidf_matrix[5] )

(6809, 28706) [0. 0. 0. ... 0. 0. 0.]


## Record Linkage using KNN

In [16]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(tfidf_matrix, test_size=0.2)

In [18]:
test.shape

(1362, 28706)

In [48]:
from sklearn.neighbors import NearestNeighbors
nbrs = NearestNeighbors(n_neighbors=2, n_jobs=-1, metric='cosine').fit(train)

In [49]:
distances, indices = nbrs.kneighbors(tfidf_matrix)

In [50]:
distances

array([[8.65256382e-01, 8.78827592e-01],
       [6.38080336e-01, 6.91841409e-01],
       [0.00000000e+00, 1.99622954e-03],
       ...,
       [4.44089210e-16, 8.58419724e-01],
       [1.11022302e-16, 6.98115164e-01],
       [0.00000000e+00, 6.30951830e-01]])

In [51]:
indices

array([[1159, 1511],
       [ 439, 1715],
       [3408,  633],
       ...,
       [3485, 3805],
       [5356, 3600],
       [4318, 2498]])

In [36]:
# For n_neighbors=1
matches = []
for i, j in enumerate(indices):
    temp = [round(distances[i][0], 2), df.iloc[i]['name'], df.iloc[j]['name'], j]
    matches.append(temp)

matches = pd.DataFrame(matches, columns=[
    'Match confidence (lower is better)',
    'Original Name',
    'Matched Name',
    'Matched Index'
])

In [37]:
matches

,Match confidence (lower is better),Original Name,Matched Name,Matched Index
0,0.87,Somewhere Else,1159 Spacious Family House in the Museum Di...,[1159]
1,0.64,Quiet Garden View Room & Super Fast Wi-Fi,439 Luxury apartment. Prime location Name: ...,[439]
2,0.00,Studio with private bathroom in the centre 1,3408 Beautiful house in the city centre of ...,[3408]
3,0.66,"Romantic, stylish B&B houseboat in canal district",4601 Big apartment surrounded by nature in ...,[4601]
4,0.28,Comfortable double room,3721 Cosy apartment in heart of Amsterdam O...,[3721]
...,...,...,...,...
6804,0.54,Romantic Houseboat in the center,1976 Authentic apartment with a view on the...,[1976]
6805,0.00,mediterranean ambiance appartment,1657 Modern & bright three-floors house clo...,[1657]
6806,0.00,Charlie + Coco's Apartment,3485 Sunny family-friendly house; garden + ...,[3485]
6807,0.00,Sleeping in a unique ship in the center of A'dam!,"5356 Small Double Room, Euphemia Hotel , ci...",[5356]


In [53]:
for i, j in enumerate(indices):
    if i < 15:
        continue
    temp = [round(distances[i][0], 2), df.iloc[i]['name'], df.iloc[j]['name'], j]
    print('Distance', round(distances[i][0], 2))
    print('Original Record')
    print(df.iloc[i]['name'])
    print(df.iloc[i]['description'])
    print('Matched Record')
    print(df.iloc[j]['name'])
    print(df.iloc[j]['description'])
    print('Matched Index', j)
    if i == 20:
        break

Distance 0.64
Original Record
Entire house with garden in De Pijp
This lovely, three story monumental house is located in De Pijp, also known as the 'Quartier Latin' of Amsterdam. It is owned by Federica and Ton, an Italian/Dutch couple with three kids.<br /><br /><b>The space</b><br />This lovely, three story monumental house is located in De Pijp, also known as the 'Quartier Latin' of Amsterdam. It is owned by Federica and Ton, an Italian/Dutch couple with three kids.<br /><br />The ground floor has a large living room with a kitchen. Two spacious doors lead into the well taken care of garden, where breakfast or dinner can be enjoyed in the sun. There is also a barbecue available. <br />On the first floor there is a bedroom with a bathroom 'en suite'. On the second floor you will find two additional bedrooms, with an additional bathroom including a bath tub. <br /><br />Features: <br />First floor: Living room, kitchen, toilet and garden<br />Second floor: Bedroom (Queen bed 140 x 20

In [73]:
df.iloc[6808][['name', 'description', 'latitude', 'longitude']].values

array(['Sunny & Bright 3 bed apt with Roof terrace',
       'This sunny, spacious,  trendy and clean 123m2  apartment is in the heart of the \'River Neighborhood (the old Jewish quarters)\'. It is the perfect place for 2 couples or a family to enjoy the beautiful city of Amsterdam!<br /><br /><b>The space</b><br />This sunny, spacious,  trendy and clean 123m2  apartment is in the heart of the \'River Neighborhood (the old Jewish quarters)\', just 5 minutes from Amsterdam Rai Convention, meaning you\'re close enough to the center and in a very nice and quiet area. For tips on where to rent bikes and the best restaurants, check our Guidebook.<br /><br />The building’s architecture is in the authentic "Amsterdamse School" Art Deco style Amsterdam is famous for. And our building block is officially a Monument! <br /><br />There are 3 bedrooms in total. There are two double bedrooms on different floors. On the main floor, there is also a kids bedroom with a 135cm crib (of which one of the s

In [74]:
df.iloc[4318][['name', 'description', 'latitude', 'longitude']].values

array(['Bright 3rd-4th floor apartment near the Foodhallen',
       'Located half a block away from the hip Foodhallen, this 3rd-4th floor apartment is very central yet extremely quiet (The street is a dead-end for traffic). The market is 2 min walk from the house and on from Mon-Sat. Public transport and all sorts of handy shops are 3 min away in the parallel Kinkerstraat.<br />The area has dozens of bars/cafes, restaurants, shops, etc. You can walk 5 min to the central canal ring.<br /><br /><b>License number</b><br />0363 6210 55B5 4B4E 8B5E',
       52.36528, 4.86515], dtype=object)

In [75]:
import math
a = df.iloc[6808]['latitude'] - df.iloc[4318]['latitude']
b = df.iloc[6808]['longitude'] - df.iloc[4318]['longitude']
math.sqrt((a*a) - (b*b))

ValueError: math domain error

In [78]:
df['latitude'].dtype

dtype('float64')

In [ ]:
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.90)]
to_drop

In [ ]:
# Drop columns id, scrape_id, maximum_minimum_nights, maximum_maximum_nights, minimum_nights_avg_ntm, maximum_nights_avg_ntm, last_scraped
columns = ['id', 
           'scrape_id', 
           'maximum_minimum_nights', 
           'maximum_maximum_nights', 
           'minimum_nights_avg_ntm', 
           'maximum_nights_avg_ntm',
          'last_scraped']
df = df.drop(columns, axis=1)
df

In [ ]:
df.dtypes

In [ ]:
numerics = ['int64', 'float64']
numeric_columns = df.select_dtypes(include=numerics).columns

In [ ]:
numeric_columns

In [ ]:
# Analyze host_listings_count
df['host_listings_count'].value_counts()